# Task-1
Нужно написать две программы: Первая генерирует бинарный файл (min 2Гб), состоящий из случайных 32-рязрядных беззнаковых целых чисел (big endian). Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.

Реализуйте две версии - 1. Простое последовательное чтение 2. Многопоточная + memory-mapped files. Сравните время работы.

In [1]:
filename = 'local_numbers_32b.npy'
size = 2*1024*1024*1024

In [2]:
def print_result(a_sum: int, a_min: int, a_max: int):
    print(f"Sum of elements: {a_sum}")
    print(f"Min element: {a_min}")
    print(f"Max element: {a_max}")

## Генерация данных
В файл записывпается массив беззнаковых чисел с байтовым порядком Bif Endian. Навазние файла и его размер задается как аргуемнты функции. 

In [3]:
from data_generation import generate_file

In [4]:
print_result(*generate_file(size, filename))

Sum of elements: 1152916250510977556
Min element: 4
Max element: 4294967286


## Последовательное чтение 1
Последовательное считывание данных и нахождение суммы, минимального и максимального чисел. Для реализации используются функции numpy.

In [5]:
import numpy as np
from data_generation import read_file

In [6]:
%%time
array = read_file(filename)
print_result(np.sum(array), np.min(array), np.max(array))

Sum of elements: 1152916250510977556
Min element: 4
Max element: 4294967286
CPU times: user 2.22 s, sys: 459 ms, total: 2.68 s
Wall time: 2.68 s


## Последовательное чтение 2
Последовательное считывание данных и нахождение суммы, минимального и максимального чисел. Реализуется стандартными средствами питона.

In [7]:
%%time
array = read_file(filename)
print_result(sum(array), min(array), max(array))

Sum of elements: 1152916250510977556
Min element: 4
Max element: 4294967286
CPU times: user 1min 52s, sys: 1.38 s, total: 1min 53s
Wall time: 1min 53s


## Последовательное чтение 3
Последовательное считывание данных и нахождение суммы, минимального и максимального чисел. Реализуется cручками.

In [8]:
def aggregation(seq, acc=0, a_min=2**32, a_max=0):
    for el in seq:
        acc, a_min, a_max = acc+el , min(a_min, el), max(a_max, el)
    return acc, a_min, a_max

In [9]:
%%time
array = read_file(filename)
print_result(*aggregation(array))

Sum of elements: 1152916250510977556
Min element: 4
Max element: 4294967286
CPU times: user 3min 32s, sys: 233 ms, total: 3min 32s
Wall time: 3min 32s


# Паралельное чтение 1
Для чтения используются memory map files. Для обработки используется пулл потоков и numpy функции

In [10]:
from data_generation import read_mmap_file
from multiprocessing.dummy import Pool as thread_pool
from toolz import juxt

In [11]:
%%time
chunked_arrays = np.array_split(read_mmap_file(size, filename), 8)
result = np.array(thread_pool(8).map(juxt(np.sum, np.min, np.max), chunked_arrays))
print_result(np.sum(result[:, 0]), np.min(result[:, 1]), np.max(result[:, 2]))

Sum of elements: 1152916250510977556
Min element: 4
Max element: 4294967286
CPU times: user 3.87 s, sys: 60.5 ms, total: 3.94 s
Wall time: 543 ms


## Паралельное чтение 2
Для чтения используются memory map files. Для обработки используется пулл потоков и встроенные фунции

In [12]:
%%time
chunked_arrays = np.array_split(read_mmap_file(size, filename), 8)
result = np.array(thread_pool(8).map(juxt(sum, min, max), chunked_arrays))
print_result(sum(result[:, 0]), min(result[:, 1]), max(result[:, 2]))

Sum of elements: 1152916250510977556
Min element: 4
Max element: 4294967286
CPU times: user 1min 44s, sys: 524 ms, total: 1min 45s
Wall time: 1min 44s


## Паралельное чтение 3
Для чтения используются memory map files. Для обработки используется пулл процессов и numpy функции

In [13]:
from multiprocessing import Pool as process_pool

In [14]:
%%time
chunked_arrays = np.array_split(read_mmap_file(size, filename), 8)
result = np.array(process_pool(8).map(juxt(np.sum, np.min, np.max), chunked_arrays))
print_result(np.sum(result[:, 0]), np.min(result[:, 1]), np.max(result[:, 2]))

Sum of elements: 1152916250510977556
Min element: 4
Max element: 4294967286
CPU times: user 883 ms, sys: 1.7 s, total: 2.58 s
Wall time: 4.09 s


## Паралельное чтение 4
Для чтения используются memory map files. Для обработки используется пулл процессов и встроенные фунции

In [ ]:
%%time
chunked_arrays = np.array_split(read_mmap_file(size, filename), 8)
result = np.array(process_pool(8).map(juxt(sum, min, max), chunked_arrays))
print_result(sum(result[:, 0]), min(result[:, 1]), max(result[:, 2]))

Наилучший результат показало решение с memory map files, пулом потоков и методами numpy. 543 ms\
Для последовательной обработки лучше всего показала себя обработка numpy функциями 2.68 s